# Counting issues by denominations

This notebook computes the number of issues by denomination for more than 50,000 records in OCRE.

## Organization of this notebook

This notebook uses Scala with the almond kernel (<https://almond.sh/>).  The following cell configures the almond kernel to find and import a series of custom libraries using syntax specific to the ammonite shell that almond use.  This is analogous to defining imports in a `build.sbt` file if you were using `sbt` to run scala.

The following section (labelled  "Analyis with generic Scala") consists of completely generic scala that could be used in any environment with access to the repositories and libraries configured in the section labelled "Notebook configuration".


## Notebook configuration

Set up notebook for access to libraries.  For reasons I don't understand (but perhaps having to do with asynchronous loading) I have to separate out the two steps of adding a maven repository and using `$ivy` imports with those repositories into separate notebook cells.

In [1]:
// 1. Add maven repository where we can find our libraries
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [10]:
// 2. Make libraries available with `$ivy` imports:
import $ivy.`edu.holycross.shot::nomisma:0.4.0`
import $ivy.`edu.holycross.shot::histoutils:2.2.0`
/*

  "edu.holycross.shot.cite" %% "xcite" % "4.1.1",
  "edu.holycross.shot" %% "ohco2" % "10.16.0",

  "edu.holycross.shot" %% "midvalidator" % "9.1.0",
  "edu.holycross.shot" %% "latincorpus" % "2.2.1",
  "edu.holycross.shot" %% "tabulae" % "6.0.1",
*/
    

import $ivy.$                                  

import $ivy.$                                     
/*

  "edu.holycross.shot.cite" %% "xcite" % "4.1.1",
  "edu.holycross.shot" %% "ohco2" % "10.16.0",

  "edu.holycross.shot" %% "midvalidator" % "9.1.0",
  "edu.holycross.shot" %% "latincorpus" % "2.2.1",
  "edu.holycross.shot" %% "tabulae" % "6.0.1",
*/
    

## Analyis with generic Scala


### Build an `Ocre` object

The following cell loads data in CEX format for:

- basic data about each issue
- texts of obverse and reverse legends
- text of OCRE description of obverse and reverse types
- geographic coordinate data for mints in OCRE

It unifies these in a single `Ocre` object 

In [3]:
import edu.holycross.shot.nomisma._

import scala.io.Source

val basicsUrl = "https://github.com/neelsmith/nomisma/raw/master/shared/src/test/resources/cex/ocre-basic-issues.cex"
// Drop header line
val data = Source.fromURL(basicsUrl).getLines.toVector.tail

val basics = for (cex <- data) yield {
    BasicIssue(cex)
}
val basicIssues = basics.toVector


val legendUrl = "https://raw.githubusercontent.com/neelsmith/nomisma/master/shared/src/test/resources/cex/ocre-legends.cex"
val legendList = for (cex <- Source.fromURL(legendUrl).getLines) yield {
    edu.holycross.shot.nomisma.Legend(cex)
}
val legends = legendList.toVector.flatten

val typesUrl = "https://raw.githubusercontent.com/neelsmith/nomisma/master/shared/src/test/resources/cex/ocre-types.cex"
val typeList = for (cex <- Source.fromURL(typesUrl).getLines) yield {
    TypeDescription(cex)
}
val typeVector = typeList.toVector.flatten

val geoUrl  = "https://raw.githubusercontent.com/neelsmith/nomisma/master/shared/src/test/resources/cex/mintgeo.cex"
val geoLines = Source.fromURL(geoUrl).getLines.toVector.tail
val mintPoints = geoLines.map(MintPoint(_)).toVector

val ocre = Ocre(basicIssues, legends, typeVector, Vector.empty[Portrait], MintPointCollection(mintPoints))




import edu.holycross.shot.nomisma._


import scala.io.Source


basicsUrl: String = "https://github.com/neelsmith/nomisma/raw/master/shared/src/test/resources/cex/ocre-basic-issues.cex"
data: Vector[String] = Vector(
  "1_2.aug.10#RIC I (second edition) Augustus 10#denarius#ar#augustus#emerita#lusitania",
  "1_2.aug.100#RIC I (second edition) Augustus 100#denarius#ar#augustus#colonia_patricia#lusitania",
  "1_2.aug.101#RIC I (second edition) Augustus 101#denarius#ar#augustus#colonia_patricia#lusitania",
  "1_2.aug.102#RIC I (second edition) Augustus 102#denarius#ar#augustus#colonia_patricia#lusitania",
  "1_2.aug.103#RIC I (second edition) Augustus 103#denarius#ar#augustus#colonia_patricia#lusitania",
  "1_2.aug.104#RIC I (second edition) Augustus 104#aureus#av#augustus#colonia_patricia#lusitania",
  "1_2.aug.105A#RIC I (second edition) Augustus 105A#denarius#ar#augustus#colonia_patricia#lusitania",
  "1_2.aug.105B#RIC I (second edition) Augustus 105B#denarius#ar#augustus#colonia_patric

### Compute and plot histogram by denomination

In [5]:
import edu.holycross.shot.histoutils._
val denomFreqs = ocre.issues.map(_.denomination).groupBy(d => d).map { case (k,v) => Frequency(k, v.size)}
val denomHisto = Histogram(denomFreqs.toVector)

import edu.holycross.shot.histoutils._

denomFreqs: collection.immutable.Iterable[Frequency[String]] = List(
  Frequency("quadrans", 262),
  Frequency("quinarius", 558),
  Frequency("double-sestertius", 9),
  Frequency("ae2", 4603),
  Frequency("ae_half_unit", 2),
  Frequency("fraction", 132),
  Frequency("hemidrachm", 3),
  Frequency("drachma", 13),
  Frequency("7-solidus", 2),
  Frequency("ar", 2),
  Frequency("aureus1_ric", 22),
  Frequency("triens", 2),
  Frequency("half_siliqua", 83),
  Frequency("reduced_half_siliqua", 1),
  Frequency("semissis", 141),
  Frequency("denarius", 6430),
  Frequency("12-as", 1),
  Frequency("multiple-gold", 238),
  Frequency("ae1", 2048),
  Frequency("multiple-silver", 50),
  Frequency("9-solidus", 8),
  Frequency("didrachm", 12),
  Frequency("reduced_siliqua", 72),
  Frequency("aureus2_ric", 24),
  Frequency("5-aureus", 1),
  Frequency("half_solidus", 11),
  Frequency("heavy_siliqua", 1),
  Frequency("large_ae2", 517),
  Frequency("as", 3404),
  Freq

In [7]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

In [8]:
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

In [11]:
val denoms = denomHisto.sorted.frequencies.map(_.item)
val counts = denomHisto.sorted.frequencies.map(_.count)
val plottable = Seq(
  Bar(
   denoms, counts
  )
)

plot(plottable)




denoms: Vector[String] = Vector(
  "antoninianus",
...
counts: Vector[Int] = Vector(
  9472,
...
plottable: Seq[Bar] = List(
  Bar(
...
res10_3: String = "plot-4cca63bc-1ace-499b-9340-4aea3fd5afe9"